# MCP Registry Demo - Part 3: Deploy MCP Registry

This notebook guides you through deploying the MCP Registry application and making it accessible.

## What You'll Learn

- How to deploy the MCP Registry
- How to configure persistent storage
- How to create services and routes
- How to verify the registry is accessible

## Step 1: Import Libraries and Setup

In [ ]:
import subprocess
import json
import os
import time
import requests
from pathlib import Path

def run_command(cmd, show_output=True):
    """Helper function to run shell commands"""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if show_output:
        if result.stdout:
            print(result.stdout)
        if result.stderr and result.returncode != 0:
            print(f"Error: {result.stderr}")
    return result.returncode, result.stdout, result.stderr

# Get repository root
repo_root = Path.cwd().parent
manifest_dir = repo_root / "manifests"
config_dir = repo_root / "config"
namespace = "mcp-registry"

print(f"Repository root: {repo_root}")
print(f"Manifest directory: {manifest_dir}")
print(f"Config directory: {config_dir}")
print(f"Target namespace: {namespace}")

## Step 2: Deploy MCP Registry

In [ ]:
print("Deploying MCP Registry...\n")

# Apply registry manifest
registry_manifest = manifest_dir / "02-mcp-registry.yaml"
returncode, stdout, stderr = run_command(f"oc apply -f {registry_manifest}")

if returncode == 0:
    print("\n✓ MCP Registry manifests applied successfully")
else:
    print("\n✗ Failed to deploy MCP Registry")
    print(f"Error: {stderr}")

## Step 3: Wait for Registry Deployment

In [ ]:
print("Waiting for MCP Registry to be ready...\n")

max_attempts = 40
attempt = 0

while attempt < max_attempts:
    returncode, stdout, stderr = run_command(
        f"oc get deployment mcp-registry -n {namespace} -o json",
        show_output=False
    )
    
    if returncode == 0:
        try:
            deployment = json.loads(stdout)
            status = deployment.get('status', {})
            ready_replicas = status.get('readyReplicas', 0)
            replicas = status.get('replicas', 0)
            
            print(f"Attempt {attempt + 1}/{max_attempts}: {ready_replicas}/{replicas} replicas ready")
            
            if ready_replicas == replicas and replicas > 0:
                print("\n✓ MCP Registry is ready!")
                break
        except json.JSONDecodeError:
            print(f"Attempt {attempt + 1}/{max_attempts}: Waiting for deployment...")
    
    attempt += 1
    time.sleep(10)
else:
    print("\n⚠ Timeout waiting for registry to be ready")
    print("Check the logs with: oc logs -n mcp-registry deployment/mcp-registry")

## Step 4: Deploy Services

In [ ]:
print("Deploying Services...\n")

# Apply services manifest
services_manifest = manifest_dir / "03-services.yaml"
returncode, stdout, stderr = run_command(f"oc apply -f {services_manifest}")

if returncode == 0:
    print("\n✓ Services created successfully")
else:
    print("\n✗ Failed to create services")
    print(f"Error: {stderr}")

## Step 5: Create Route for External Access

In [ ]:
print("Creating Route for external access...\n")

# Apply route manifest
route_manifest = manifest_dir / "04-route.yaml"
returncode, stdout, stderr = run_command(f"oc apply -f {route_manifest}")

if returncode == 0:
    print("\n✓ Route created successfully")
else:
    print("\n✗ Failed to create route")
    print(f"Error: {stderr}")

## Step 6: Get Registry URL

In [ ]:
print("Getting Registry URL...\n")

returncode, stdout, stderr = run_command(
    f"oc get route mcp-registry -n {namespace} -o json",
    show_output=False
)

if returncode == 0:
    try:
        route = json.loads(stdout)
        host = route['spec']['host']
        tls = route['spec'].get('tls', {})
        protocol = 'https' if tls else 'http'
        registry_url = f"{protocol}://{host}"
        
        print(f"✓ MCP Registry URL: {registry_url}")
        print(f"\nYou can access the registry at: {registry_url}")
        
        # Save URL for later use
        os.environ['MCP_REGISTRY_URL'] = registry_url
    except (json.JSONDecodeError, KeyError) as e:
        print(f"✗ Error parsing route: {e}")
else:
    print("✗ Could not retrieve route information")

## Step 7: Verify Registry Status

In [ ]:
print("Checking all deployed resources...\n")

# Check PVC
print("Persistent Volume Claim:")
run_command(f"oc get pvc -n {namespace}")

print("\nConfigMaps:")
run_command(f"oc get configmap -n {namespace}")

print("\nDeployments:")
run_command(f"oc get deployment -n {namespace}")

print("\nPods:")
run_command(f"oc get pods -n {namespace}")

print("\nServices:")
run_command(f"oc get svc -n {namespace}")

print("\nRoutes:")
run_command(f"oc get route -n {namespace}")

## Step 8: Test Registry Connectivity

In [ ]:
print("Testing registry connectivity...\n")

registry_url = os.environ.get('MCP_REGISTRY_URL')

if registry_url:
    try:
        # Test health endpoint
        health_url = f"{registry_url}/health"
        print(f"Testing: {health_url}")
        
        response = requests.get(health_url, timeout=10, verify=False)
        
        if response.status_code == 200:
            print(f"✓ Health check passed: {response.json()}")
        else:
            print(f"⚠ Health check returned status {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"⚠ Could not connect to registry: {e}")
        print("The registry may still be starting up. Try again in a few moments.")
else:
    print("✗ Registry URL not available")

## Step 9: View Registry Logs

In [ ]:
print("Fetching registry logs (last 30 lines)...\n")

returncode, stdout, stderr = run_command(
    f"oc logs -n {namespace} deployment/mcp-registry --tail=30"
)

if returncode != 0:
    print("\n⚠ Could not fetch logs. The registry may not be running yet.")

## Summary

In this notebook, you:
- Deployed the MCP Registry with persistent storage
- Created services for internal access
- Created a route for external access
- Verified the registry is accessible

### Next Steps

Continue to **04-manage-and-test.ipynb** to:
- Add tools to the registry
- Test registry operations
- Explore management features

### Troubleshooting

If the registry is not accessible:
1. Check logs: `oc logs -n mcp-registry deployment/mcp-registry`
2. Check pod status: `oc get pods -n mcp-registry`
3. Check events: `oc get events -n mcp-registry`
4. Verify PVC is bound: `oc get pvc -n mcp-registry`
5. Test internal service: `oc port-forward -n mcp-registry svc/mcp-registry 8080:8080`